In [1]:
from pytorch_fid import fid_score

import os
from skimage.metrics import structural_similarity as ssim
from skimage import io
from skimage.color import rgb2gray
import numpy as np
import cv2
import numpy as np
import os
from math import log10, sqrt

import tqdm 
import pandas as pd 
from pytorch_fid.fid_score import InceptionV3, calculate_frechet_distance, compute_statistics_of_path,calculate_fid_given_paths
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from PIL import Image

def get_img_metric(img_dir, img_w_dir, num_imgs=None):
    filenames = os.listdir(img_dir)
    filenames.sort()
    if num_imgs is not None:
        filenames = filenames[:num_imgs]
    log_stats = []
    for ii, filename in enumerate(tqdm.tqdm(filenames)):
        pil_img_ori = Image.open(os.path.join(img_dir, filename)).convert('RGB')
        pil_img_ori = pil_img_ori.resize((512, 512), Image.BICUBIC)
        pil_img = Image.open(os.path.join(img_w_dir, filename))
        pil_img = pil_img.resize((512, 512))
        
        img_ori = np.asarray(pil_img_ori)
        img = np.asarray(pil_img)
        #print(img_ori.shape, img.shape)
        log_stat = {
            'filename': filename,
            'ssim': structural_similarity(img_ori, img, channel_axis=2),
            'psnr': peak_signal_noise_ratio(img_ori, img),
            'linf': np.amax(np.abs(img_ori.astype(int)-img.astype(int)))
        }
        log_stats.append(log_stat)
    return log_stats

/hhd2/hqq/anaconda3/envs/WM_env/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /hhd2/hqq/anaconda3/envs/WM_env/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")
/hhd2/hqq/anaconda3/envs/WM_env/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# img_dir= "ckpts/0/test/original_imgs"
# img_w_dir = "ckpts/0/test/watermarked_imgs"
#params.num_imgs = 10000

img_dir= "datasets/test_WikiArt3"
img_w_dir = "datasets/test_WikiArt3_pgd/noise-ckpt/50"
img_metrics = get_img_metric(img_dir, img_w_dir)
img_df = pd.DataFrame(img_metrics)

100%|██████████| 16/16 [00:01<00:00, 10.12it/s]


In [18]:
fid= calculate_fid_given_paths([img_w_dir,"ckpts/2/test/original_imgs"],batch_size=1,device='cuda',dims=2048)
print(fid/16)  ##换成除以文件夹中的图片个数

100%|██████████| 192/192 [00:02<00:00, 77.35it/s]


19.297342927002514


In [9]:
output_dir = 'datasets/test_WikiArt3_pgd/noise-ckpt/50/test'
os.makedirs(output_dir,exist_ok=True)
img_df.to_csv(os.path.join(output_dir, 'img_metrics2.csv'), index=False)
ssims = img_df['ssim'].tolist()
psnrs = img_df['psnr'].tolist()
linfs = img_df['linf'].tolist()
ssim_mean, ssim_std, ssim_max, ssim_min = np.mean(ssims), np.std(ssims), np.max(ssims), np.min(ssims) 
psnr_mean, psnr_std, psnr_max, psnr_min = np.mean(psnrs), np.std(psnrs), np.max(psnrs), np.min(psnrs)
linf_mean, linf_std, linf_max, linf_min = np.mean(linfs), np.std(linfs), np.max(linfs), np.min(linfs)
print(f"SSIM: {ssim_mean:.4f}±{ssim_std:.4f} [{ssim_min:.4f}, {ssim_max:.4f}]")
print(f"PSNR: {psnr_mean:.4f}±{psnr_std:.4f} [{psnr_min:.4f}, {psnr_max:.4f}]")
print(f"Linf: {linf_mean:.4f}±{linf_std:.4f} [{linf_min:.4f}, {linf_max:.4f}]")

SSIM: 0.3002±0.1050 [0.0756, 0.5594]
PSNR: 14.9174±2.8831 [7.9690, 18.7515]
Linf: 229.0000±23.8982 [169.0000, 255.0000]


In [19]:
img_dir= "datasets/test_WikiArt3"
img_w_dir = "datasets/test_WikiArt3_pgd/noise-ckpt/10"
img_metrics = get_img_metric(img_dir, img_w_dir)
img_df = pd.DataFrame(img_metrics)

fid= calculate_fid_given_paths([img_w_dir,"ckpts/2/test/original_imgs" ],batch_size=32,device='cuda',dims=2048)
print(fid/16)  ##换成除以文件夹中的图片个数

output_dir = 'datasets/test_WikiArt3_pgd/noise-ckpt/10/test'
os.makedirs(output_dir,exist_ok=True)
img_df.to_csv(os.path.join(output_dir, 'img_metrics2.csv'), index=False)
ssims = img_df['ssim'].tolist()
psnrs = img_df['psnr'].tolist()
linfs = img_df['linf'].tolist()
ssim_mean, ssim_std, ssim_max, ssim_min = np.mean(ssims), np.std(ssims), np.max(ssims), np.min(ssims) 
psnr_mean, psnr_std, psnr_max, psnr_min = np.mean(psnrs), np.std(psnrs), np.max(psnrs), np.min(psnrs)
linf_mean, linf_std, linf_max, linf_min = np.mean(linfs), np.std(linfs), np.max(linfs), np.min(linfs)
print(f"SSIM: {ssim_mean:.4f}±{ssim_std:.4f} [{ssim_min:.4f}, {ssim_max:.4f}]")
print(f"PSNR: {psnr_mean:.4f}±{psnr_std:.4f} [{psnr_min:.4f}, {psnr_max:.4f}]")
print(f"Linf: {linf_mean:.4f}±{linf_std:.4f} [{linf_min:.4f}, {linf_max:.4f}]")

100%|██████████| 16/16 [00:01<00:00, 10.70it/s]


100%|██████████| 6/6 [00:02<00:00,  2.40it/s]


19.29539641623253
SSIM: 0.2993±0.1052 [0.0753, 0.5625]
PSNR: 14.9137±2.8823 [7.9709, 18.7471]
Linf: 228.3125±24.8765 [167.0000, 255.0000]
